In [1]:
######## 1 . MOUNT TO AWS S3 ######

In [2]:
# mount s3 to dbfs
ACCESS_KEY = ***********
SECRET_KEY = ***********
ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")
AWS_BUCKET_NAME = ***********
MOUNT_NAME = "wildFire-Project"
dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)
display(dbutils.fs.ls("/mnt/%s" % MOUNT_NAME))

path,name,size
dbfs:/mnt/wildFire-Project/1.0/,1.0/,0
dbfs:/mnt/wildFire-Project/1.1/,1.1/,0
dbfs:/mnt/wildFire-Project/2.0 beta/,2.0 beta/,0
dbfs:/mnt/wildFire-Project/2.1 beta/,2.1 beta/,0
dbfs:/mnt/wildFire-Project/2.1 beta forrest only/,2.1 beta forrest only/,0
dbfs:/mnt/wildFire-Project/2.1 beta forrest only _ Yanxiang Testing/,2.1 beta forrest only _ Yanxiang Testing/,0
dbfs:/mnt/wildFire-Project/2.2 beta forrest only/,2.2 beta forrest only/,0
dbfs:/mnt/wildFire-Project/2.3 beta burn scar only/,2.3 beta burn scar only/,0
dbfs:/mnt/wildFire-Project/2.3 beta burn scar only mse/,2.3 beta burn scar only mse/,0
dbfs:/mnt/wildFire-Project/2.4 burn scar only/,2.4 burn scar only/,0


In [3]:
######## 2. DATA lOADING ######

In [4]:
sample_img_dir = "/mnt/" + MOUNT_NAME + "/image_update"
sample_mask_dir = "/mnt/" + MOUNT_NAME + "/Label_Update"

In [5]:
######## 3. DATA PREPROCESSING ######

In [6]:
from pyspark.ml.image import ImageSchema
image_df = ImageSchema.readImages(sample_img_dir)
mask_df = ImageSchema.readImages(sample_mask_dir)
image_df = image_df.orderBy('image.origin', ascending=True)
mask_df = mask_df.orderBy('image.origin', ascending=True)

/databricks/spark/python/pyspark/ml/image.py:236: DeprecationWarning: `ImageSchema.readImage` is deprecated. Use `spark.read.format("image").load(path)` instead.
 "Use `spark.read.format(\"image\").load(path)` instead.", DeprecationWarning)

In [7]:
######## 3.1 Data PACKGING & SHUFFLING ######

In [8]:
import numpy as np
import random
random_list = list(range(129))
random.shuffle(random_list)

ids = image_df.count()
ims = mask_df.count()

im_width = 720
im_height = 480

row_image = image_df.collect()
row_mask  = mask_df.collect()
image_arr = np.zeros((129, im_height, im_width, 3), dtype=np.uint8)
mask_arr = np.zeros((129, im_height, im_width, 3), dtype=np.uint8)
for i in range(129):
  image_arr[i] = ImageSchema.toNDArray(row_image[i].image) 
for i in range(129):
  mask_arr[i] = ImageSchema.toNDArray(row_mask[i].image)
  
k = 0
image_arr_1 = np.zeros((129, im_height, im_width, 3), dtype=np.uint8)
mask_arr_1 = np.zeros((129, im_height, im_width, 3), dtype=np.uint8)
for i in random_list:
  image_arr_1[k] = image_arr[i]
  mask_arr_1[k]  = mask_arr[i] 
  k = k + 1

In [9]:
######## 3.2 DATA PREPROCESSING VARIFICATION ######

In [10]:
import matplotlib.pyplot as plt

plt.imshow(image_arr_1[125])

In [11]:
plt.imshow(mask_arr_1[125])

In [12]:
######## 3.3 DATA CONVERSION ######

In [13]:
# convert data to the 3-D array format
X_data  = image_arr_1
label  = mask_arr_1[:,:,:,2]
y_data = label.reshape((label.shape[0], label.shape[1], label.shape[2], 1))
X_data =X_data/ np.max(X_data.astype('float'))
y_data =y_data/ np.max(y_data.astype('float'))

ratio = 0.9  
X_train = X_data[:round(129*ratio)]
y_train = y_data[:round(129*ratio)]
X_test = X_data[round(129*ratio)+1:-1]
Y_test = y_data[round(129*ratio)+1:-1]

In [14]:
######## 4. MODELING ######

In [15]:
from keras.datasets import mnist
from keras.layers import  Input,Conv2D,Conv3D,BatchNormalization,Activation,Conv2DTranspose, Conv3DTranspose, MaxPooling2D, MaxPooling3D, concatenate
from keras.models import Model, load_model
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import Adam
import keras.backend as K
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import mlflow
import mlflow.keras
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.
/databricks/python/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping, MutableMapping

In [16]:
######## 4.1 MODEL SETUP ######

In [17]:
MODEL_VERSION = '2.5' 
MODEL_NAME = 'Unet'
model_save_path = '/dbfs/mnt/'+MOUNT_NAME+'/'+MODEL_VERSION+'/'+MODEL_NAME
model_Unet = mlflow.keras.load_model(model_save_path)
callbacks = [ModelCheckpoint('model.h5', verbose = 1, save_best_only = True, save_weights_only = True)]

In [18]:
# Create image generator
data_gen_args = dict(rotation_range=180,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=[0.5, 1.5]
                     )
image_datagen = ImageDataGenerator(**data_gen_args)

seed = 1

def XYaugmentGenerator(X1, y, seed, batch_size):
    genX1 = image_datagen.flow(X1, y, batch_size=batch_size, seed=seed)
    genX2 = image_datagen.flow(y, X1, batch_size=batch_size, seed=seed)
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        yield X1i[0], X2i[0]


In [19]:
img_rows, img_cols = im_height, im_width
num_convs_per_layer=2
image_channels=1
num_dim=2
batch_norm=0
num_chans=3
feat_size_0=16
filt_size = (4,4)
max_pooling_size=(2,2)
U_net_depth=4
feat_size_growth=1.5
batchsize=1
feat_size = [int(feat_size_0 * feat_size_growth ** i) for i in range(U_net_depth + 1)]  



def model():
    def Unet_block( feat_size, filt_size, num_convs_per_layer, inputs):
        for i_conv in range(num_convs_per_layer):
            layer=Conv2D(feat_size, kernel_size=filt_size, padding='same', activation='relu')(inputs)     
        if batch_norm:
            return BatchNormalization()(layer)
        else:
            return layer
    
    def encode(inputs):
        unet_layer = [[] for i in range(U_net_depth+1)]
        unet_layer[0] = inputs
        for layer in range(U_net_depth):
            unet_layer[layer] = Unet_block(feat_size[layer], filt_size, num_convs_per_layer, unet_layer[layer])
            unet_layer[layer + 1] = MaxPooling2D(pool_size=max_pooling_size)(unet_layer[layer])
        return unet_layer
    
    def decode(unet_layer):
        for layer in range(U_net_depth, 0, -1):
            unet_layer[layer] = Unet_block(feat_size[layer], filt_size, num_convs_per_layer, unet_layer[layer])
            trans_layer = Conv2DTranspose(feat_size[layer - 1], kernel_size=filt_size, strides=max_pooling_size, padding='same', activation='relu')(unet_layer[layer])
            if K.int_shape(trans_layer)[1]==K.int_shape(unet_layer[layer - 1])[1]:
                unet_layer[layer - 1] = concatenate([trans_layer, unet_layer[layer - 1]])
            else:
                K.resize_images(trans_layer,K.int_shape(unet_layer[layer - 1])[1], K.int_shape(unet_layer[layer - 1])[2], data_format='channels_last', interpolation='nearest' )
                unet_layer[layer - 1] = concatenate([trans_layer, unet_layer[layer - 1]])
        unet_layer[0] = Unet_block(feat_size[0], filt_size, num_convs_per_layer, unet_layer[0])
        return unet_layer[0]
        
    inputs = Input(shape=(img_rows, img_cols, num_chans))        
    last_layer = decode(encode(inputs))                                                                       
    last_layer = Conv2D(1, (1, 1), activation='sigmoid')(last_layer)
    return Model(inputs=[inputs], outputs=[last_layer])

In [20]:
######## 4.2 TRAINING ######

In [21]:
model_Unet=model()
model_Unet.summary()
model_Unet.compile(loss='binary_crossentropy', optimizer='Adam')
model_Unet.fit(X_train, y_train, epochs=5, batch_size=batchsize,  validation_data=(X_test,Y_test))

Model: "model_1"
__________________________________________________________________________________________________
Layer (type) Output Shape Param # Connected to 
==================================================================================================
input_1 (InputLayer) (None, 480, 720, 3) 0 
__________________________________________________________________________________________________
conv2d_1 (Conv2D) (None, 480, 720, 16) 784 input_1[0][0] 
__________________________________________________________________________________________________
conv2d_2 (Conv2D) (None, 480, 720, 16) 4112 conv2d_1[0][0] 
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D) (None, 240, 360, 16) 0 conv2d_2[0][0] 
__________________________________________________________________________________________________
conv2d_3 (Conv2D) (None, 240, 360, 24) 6168 max_pooling2d_1[0][0] 
__________________________________________________________________________________________________
conv2d_4 (Conv2D) (None, 240, 360, 24) 9240 conv2d_3[0][0] 
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D) (None, 120, 180, 24) 0 conv2d_4[0][0] 
__________________________________________________________________________________________________
conv2d_5 (Conv2D) (None, 120, 180, 36) 13860 max_pooling2d_2[0][0] 
__________________________________________________________________________________________________
conv2d_6 (Conv2D) (None, 120, 180, 36) 20772 conv2d_5[0][0] 
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D) (None, 60, 90, 36) 0 conv2d_6[0][0] 
__________________________________________________________________________________________________
conv2d_7 (Conv2D) (None, 60, 90, 54) 31158 max_pooling2d_3[0][0] 
__________________________________________________________________________________________________
conv2d_8 (Conv2D) (None, 60, 90, 54) 46710 conv2d_7[0][0] 
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D) (None, 30, 45, 54) 0 conv2d_8[0][0] 
__________________________________________________________________________________________________
conv2d_9 (Conv2D) (None, 30, 45, 81) 70065 max_pooling2d_4[0][0] 
__________________________________________________________________________________________________
conv2d_10 (Conv2D) (None, 30, 45, 81) 105057 conv2d_9[0][0] 
__________________________________________________________________________________________________
conv2d_transpose_1 (Conv2DTrans (None, 60, 90, 54) 70038 conv2d_10[0][0] 
__________________________________________________________________________________________________
concatenate_1 (Concatenate) (None, 60, 90, 108) 0 conv2d_transpose_1[0][0] 
 conv2d_8[0][0] 
__________________________________________________________________________________________________
conv2d_11 (Conv2D) (None, 60, 90, 54) 93366 concatenate_1[0][0] 
__________________________________________________________________________________________________
conv2d_12 (Conv2D) (None, 60, 90, 54) 46710 conv2d_11[0][0] 
__________________________________________________________________________________________________
conv2d_transpose_2 (Conv2DTrans (None, 120, 180, 36) 31140 conv2d_12[0][0] 
__________________________________________________________________________________________________
concatenate_2 (Concatenate) (None, 120, 180, 72) 0 conv2d_transpose_2[0][0] 
 conv2d_6[0][0] 
__________________________________________________________________________________________________
conv2d_13 (Conv2D) (None, 120, 180, 36) 41508 concatenate_2[0][0] 
__________________________________________________________________________________________________
conv2d_14 (Conv2D) (None, 120, 180, 36) 20772 conv2d_13[0][0] 
____________________

In [22]:
######## 5. DATA POSTPROCESSING ######

In [23]:
preds_train = model_Unet.predict(X_train, verbose=1)
preds_val = model_Unet.predict(X_test, verbose=1)


32/116 [=======>......................] - ETA: 43s 64/116 [===============>..............] - ETA: 26s 96/116 [=======================>......] - ETA: 10s116/116 [==============================] - 60s 514ms/step
11/11 [==============================] - 6s 512ms/step

In [24]:
preds_train_t = (preds_train > 0.25).astype(np.uint8)
preds_val_t = (preds_val > 0.25).astype(np.uint8)

In [25]:
######## 6. RESULT VISUALIZATION ######

In [26]:
def conv_float_int(image):
    return ((image-np.min(image))/(np.max(image)-np.min(image))*255).astype(int)

In [27]:
import cv2

# visualization
def plot_sample(X, y, preds, binary_preds, ix=None):
    """Function to plot the results"""
    if ix is None:
        ix = random.randint(0, len(X))

    has_mask = y[ix].max() > 0

    fig, ax = plt.subplots(1, 4, figsize=(20, 10))
    ax[0].imshow(conv_float_int(X[ix]))
    if has_mask:
        ax[0].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[0].set_title('Raw')
    ax[1].imshow(conv_float_int(y[ix, :, :, 0]))
    ax[1].set_title('mask')

    ax[2].imshow(conv_float_int(preds[ix, :, :, 0]), vmin=0, vmax=255)
    if has_mask:
        ax[2].contour(conv_float_int(y[ix, :, :, 0]), colors='k', levels=[0.5])
    ax[2].set_title('mask Predicted')
    ret2,th1 = cv2.threshold(conv_float_int(preds[ix, :, :, 0]).astype('uint8'), 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    ax[3].imshow(conv_float_int(th1), vmin=0, vmax=255)
    if has_mask:
        ax[3].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[3].set_title('mask Predicted binary');

In [28]:
plot_sample(X_test, Y_test, preds_val, preds_val_t, ix=5)

In [29]:
######## 7. MODEL DEPLOYMENT ######

In [30]:
# save model to aws s3
import mlflow
import mlflow.keras
MODEL_VERSION = '2.6'
MODEL_NAME = 'Unet'
model_save_path = '/dbfs/mnt/'+MOUNT_NAME+'/'+MODEL_VERSION+'/'+MODEL_NAME

mlflow.keras.save_model(model_Unet, model_save_path)

In [31]:
######## 8. UNMOUNT FROM AWS S3 ######

In [32]:
# unmount from s3
dbutils.fs.unmount("/mnt/" + MOUNT_NAME)

/mnt/wildFire-Project has been unmounted.
Out[42]: True